### Calculating Null Distribution

Gregory Way, modified from code written by Adeniyi Adeboye

- Null distribution - is generated by getting the median correlation score of randomly combined compounds that do not share/come from the same MOAs.

### The goal here:

- is to compute the p-value for each MOA per dose by evaluating the probability of random combinations of compounds (from different MOAs) having greater median correlation score than compounds of the same MOA.
- In our case, we generated 1000 median correlation scores from randomly combined compounds as the **null distribution** for each MOA_SIZE class ***i.e. for a moa_size class - we have 1000 medians scores from randomly combined compounds of different MOAs.***
- Moa_size is the number of compounds in a specific MOA and moa_size class is a specific group of MOAs that have the same number of compounds ***e.g all MOAs with just 2 compounds in them are in the same moa_size class.***

In [1]:
import os
import pathlib
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from pycytominer import feature_select
from statistics import median
import random
sns.set_style("darkgrid")
from scipy import stats
import pickle
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

In [2]:
np.random.seed(42)

#### - Load in the datasets required, 

- They were generated from the `cell_painting_moa_median_scores_calculation notebook`

In [3]:
data_moa = pd.read_csv(os.path.join('moa_sizes_consensus_datasets', 'modz_consensus_data.csv'))
data_moa_vals = pd.read_csv(os.path.join('moa_sizes_consensus_datasets', 'matching_score_per_MOA_CellPainting_dose_independent.tsv.gz'), sep="\t")
data_moa_cpds = pd.read_csv(os.path.join('moa_sizes_consensus_datasets', 'cellpainting_moa_compounds.csv'))
data_moa_cpd_summary = pd.read_csv(os.path.join('moa_sizes_consensus_datasets', 'cellpainting_moa_compounds_dose_independent.csv'))

In [4]:
data_moa.shape

(8322, 1032)

In [5]:
data_moa_vals.shape

(211, 3)

In [6]:
data_moa_cpds.shape

(211, 8)

In [7]:
def get_cpd_agg(data_moa, dose_number):
    """
    This function calculates the mean value of distinct compounds
    """
    
    df_dose = data_moa[data_moa['Metadata_dose_recode'] == dose_number].copy()
    meta_cols = [col for col in df_dose.columns.tolist() 
                 if (col.startswith("Metadata_"))]
    df_dose.drop(meta_cols, axis = 1, inplace = True)
    df_compound_agg = df_dose.groupby(['pert_iname']).agg(['mean'])
    df_compound_agg.columns  = df_compound_agg.columns.droplevel(1)
    df_compound_agg.rename_axis(None, axis=0, inplace = True)
    
    return df_compound_agg

In [8]:
def cpds_found_in_all_doses(data_moa):
    """This function return a list of compounds found in all doses (1 - 6)"""
    cpds_fd = []
    for num in range(1,7):
        df_cpd_agg = get_cpd_agg(data_moa, num)
        all_cpds = df_cpd_agg.index.tolist()
        cpds_fd.append(all_cpds)
    
    cpds_fd_in_all = [cpd for list_cpds in cpds_fd 
                      for cpd in list_cpds 
                      if all(cpd in list_of_cpds for list_of_cpds in cpds_fd)]
    cpds_fd_in_all = list(set(cpds_fd_in_all))
    
    return cpds_fd_in_all

In [9]:
# Select only the same compounds as previous calculations
cpds_fd_in_all = cpds_found_in_all_doses(data_moa)

In [10]:
len(cpds_fd_in_all)

1327

In [11]:
all_moa_list = data_moa['moa'].unique().tolist()

In [12]:
len(all_moa_list)

583

In [13]:
all_moa_dict = {moa: [cpd for cpd in data_moa['pert_iname'][data_moa['moa']== moa].unique().tolist() 
                      if cpd in cpds_fd_in_all]
                for moa in all_moa_list}
all_moa_dict = {kys:all_moa_dict[kys] for kys in all_moa_dict if all_moa_dict[kys]}

In [14]:
len(all_moa_dict)

583

In [15]:
len(data_moa_vals.moa.unique())

211

In [16]:
data_moa_cpds = data_moa_cpds.drop(columns=["moa_size"]).merge(data_moa_vals, on="moa")

In [17]:
def generate_moa_size_dict(df_moa_cpds):
    """
    Generates a dictionary with distinct moa_sizes 
    (moa_size == number of compounds that is present in each MOA) 
    as the keys and all compounds of MOAs with that particular size as the values
    """
    moa_size_dict = {}
    for size in df_moa_cpds['no_of_replicates'].unique():
        size_df = df_moa_cpds[df_moa_cpds['no_of_replicates'] == size].drop(['no_of_replicates', 'moa', 'spearman_correlation'], axis = 1)
        size_df_values = size_df.values.tolist()
        size_df_values = [x.split(";") for cpd_list in size_df_values for x in cpd_list]
        size_df_values = list(set([x for cpd in size_df_values for x in cpd]))
        moa_size_dict[size] = size_df_values
    return moa_size_dict

In [18]:
moa_sizes_dict = generate_moa_size_dict(data_moa_cpds)

In [19]:
len(data_moa_cpds['no_of_replicates'].unique())

21

In [20]:
len(moa_sizes_dict)

21

In [21]:
def get_random_cpds(all_cpds, moa_size, moa_cpds, all_moa_cpds):
    """
    This function return a list of random cpds that are not of the same moas 
    or found in the current moa cpd's list
    """
    while (True):
        random_cpds = random.sample(all_cpds, moa_size)
        if not (any(cpds in moa_cpds for cpds in random_cpds)):
            break
    return random_cpds

#### - You only need to generate the null distribution once (i.e. you can re-use the pickled null distribution for other consensus data), since the 1000 lists of randomly generated compounds combinations  for each MOA are found in all doses and all consensus datasets

In [22]:
def get_null_distribution_cpds(moa_size_dict, cpds_list, all_moa_dict, rand_num = 1000):
    
    """
    This function returns the null distribution dictionary, with MOAs as the keys and 
    1000 lists of randomly selected compounds combinations as the values for each MOA
    """
    null_distribution_moa = {}
    for size in moa_size_dict:
        print(size)
        moa_cpds = moa_size_dict[size]
        moa_cpds_list = []
        for idx in range(rand_num):
            start_again = True
            while (start_again):
                rand_cpds = get_random_cpds(cpds_list, size, moa_cpds, all_moa_dict)
                if rand_cpds not in moa_cpds_list:
                    start_again = False
            moa_cpds_list.append(rand_cpds)
        null_distribution_moa[size] = moa_cpds_list
    
    return null_distribution_moa

In [23]:
null_distribution_moa = get_null_distribution_cpds(moa_sizes_dict, cpds_fd_in_all, all_moa_dict)

12
18
48
120
24
180
30
6
84
66
90
42
150
54
108
72
36
78
138
168
60


In [24]:
#save the null_distribution_moa to pickle, you only need to run the code once
with open(os.path.join('moa_sizes_consensus_datasets', 'null_distribution_doseindependent.pickle'), 'wb') as handle:
    pickle.dump(null_distribution_moa, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
##load the null_distribution_moa from pickle
with open(os.path.join('moa_sizes_consensus_datasets', 'null_distribution_doseindependent.pickle'), 'rb') as handle:
    null_distribution_moa = pickle.load(handle)

In [26]:
print('moa_size', '\tnumber of generated lists of randomly combined compounds')
for keys in null_distribution_moa:
    print(keys, '\t\t', len(null_distribution_moa[keys]))

moa_size 	number of generated lists of randomly combined compounds
12 		 1000
18 		 1000
48 		 1000
120 		 1000
24 		 1000
180 		 1000
30 		 1000
6 		 1000
84 		 1000
66 		 1000
90 		 1000
42 		 1000
150 		 1000
54 		 1000
108 		 1000
72 		 1000
36 		 1000
78 		 1000
138 		 1000
168 		 1000
60 		 1000


In [27]:
def assert_null_distribution(null_distribution_moa):
    
    """
    This function assert that each of the list in the 1000 lists of 
    random compounds combination for each MOA are distinct with no duplicates
    """
    
    duplicates_moa = {}
    for keys in null_distribution_moa:
        null_dist = null_distribution_moa[keys]
        for cpds_moa in null_dist:
            cpds_duplicates = []
            new_list = list(filter(lambda cpds_list: cpds_list != cpds_moa, null_dist))
            if (len(new_list) != len(null_dist) - 1):
                cpds_duplicates.append(cpds_moa)
        if cpds_duplicates:
            duplicates_moa[keys] = cpds_duplicates
    return duplicates_moa

In [ ]:
duplicates_cpds_list = assert_null_distribution(null_distribution_moa)

In [ ]:
duplicates_cpds_list ##no duplicate found

{}

In [54]:
def get_cpd_agg_dose_independent(data_moa):
    """
    This function calculates the mean value of distinct compounds
    """
    
    df = data_moa.copy()

    df_cpd_agg = df.groupby(['pert_iname', 'Metadata_dose_recode']).agg(['mean']).reset_index()
    df_cpd_agg.index = df_cpd_agg.pert_iname
    
    meta_cols = ['pert_iname', "Metadata_dose_recode", "Metadata_mmoles_per_liter"]
        
    df_cpd_agg.drop(meta_cols, axis = 1, inplace = True)
    
    return df_cpd_agg

In [59]:
def calc_null_dist_median_scores(data_moa, moa_cpds_list):
    """
    This function calculate the median of the correlation 
    values for each of the list in the 1000 lists of 
    random compounds combination for each MOA
    """
    df_cpd_agg = get_cpd_agg_dose_independent(data_moa)
    median_corr_list = []
    for list_of_cpds in moa_cpds_list:
        df_cpds = df_cpd_agg.loc[list_of_cpds]
        cpds_corr = df_cpds.transpose().corr(method='spearman')
        
        if len(list_of_cpds) == 1:
            median_corr_val = 1
        else:
            cpds_corr.index.name = "pert_iname_compare"
            cpds_corr = cpds_corr.reset_index().melt(id_vars = "pert_iname_compare", value_name="spearman_corr")
            cpds_corr = cpds_corr.assign(keep_me_diff_comparison = cpds_corr.pert_iname_compare != cpds_corr.pert_iname)
            cpds_corr = cpds_corr.query("keep_me_diff_comparison")
            median_corr_val = cpds_corr.spearman_corr.median()

        median_corr_list.append(median_corr_val)
    return median_corr_list

**A P value can be computed nonparametrically by evaluating the probability of random compounds of different MOAs having greater median similarity value than compounds of the same MOAs.**

In [60]:
def get_p_value(median_scores_list, df_moa_values, moa_name):
    """
    This function calculate the p-value from the 
    null_distribution median scores for each MOA
    """
    actual_med = df_moa_values.loc[moa_name, :]
    p_value = np.sum(median_scores_list >= actual_med) / len(median_scores_list)
    return p_value

In [61]:
def get_null_dist_median_scores(null_distribution_moa, df_moa):
    """ 
    This function calculate the median correlation scores for all 
    1000 lists of randomly combined compounds for each moa_size class 
    """
    null_distribution_medians = {}
    for key in null_distribution_moa:
        null_distribution_medians[key] = calc_null_dist_median_scores(df_moa, null_distribution_moa[key])
    return null_distribution_medians

In [62]:
null_distribution_medns = get_null_dist_median_scores(null_distribution_moa, data_moa)

/Users/gway/miniconda3/envs/lincs-complimentarity/lib/python3.9/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
/Users/gway/miniconda3/envs/lincs-complimentarity/lib/python3.9/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
/Users/gway/miniconda3/envs/lincs-complimentarity/lib/python3.9/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
/Users/gway/miniconda3/envs/lincs-complimentarity/lib/python3.9/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted

In [63]:
def transform_dataframe(rep, rep_name):
    """
    Transforms replicate correlation dataframe to have 3 columns: 
    dose, correlation_values and type of replicates
    """
    df_reps = pd.DataFrame.from_dict(rep, orient='index').T
    rep_melt = df_reps.melt(var_name="dose", value_name="correlation_values")
    rep_melt['type'] = rep_name
    return rep_melt

In [64]:
threshold_df = []
for n_replicate in null_distribution_medns.keys():
    matched_null = pd.DataFrame(null_distribution_medns[n_replicate])
    thresh = matched_null.quantile(0.95).values[0]
    threshold_df.append([n_replicate, "all_doses", thresh])
        
threshold_df = pd.DataFrame(threshold_df, columns=["n_replicates", "dose", "95th_threshold"])
threshold_df

,n_replicates,dose,95th_threshold
0,12,all_doses,0.012974
1,18,all_doses,0.010977
2,48,all_doses,0.008786
3,120,all_doses,0.007790
4,24,all_doses,0.010778
5,180,all_doses,0.007625
6,30,all_doses,0.009852
7,6,all_doses,0.018276
8,84,all_doses,0.007934
9,66,all_doses,0.008390


In [65]:
def get_p_value(median_scores_list, df_moa_values, moa_name):
    """
    This function calculate the p-value from the 
    null_distribution median scores for each MOA
    """
    actual_med = df_moa_values.loc[moa_name, :].spearman_correlation
    p_value = np.sum(median_scores_list >= actual_med) / len(median_scores_list)
    return p_value

In [66]:
def get_moa_p_vals(null_dist_median, df_moa_values):
    """
    This function returns a dict, with MOAs as the keys and the MOA's p-values as the values
    """
    null_p_vals = {}
    df_moa_values = df_moa_values.set_index('moa').rename_axis(None, axis=0)
    for key in null_dist_median:
        df_moa_size = df_moa_values[df_moa_values['no_of_replicates'] == key]
        for moa in df_moa_size.index:
            moa_p_value = get_p_value(null_dist_median[key], df_moa_size, moa)
            null_p_vals[moa] = moa_p_value
    sorted_null_p_vals = {key:value for key, value in sorted(null_p_vals.items(), key=lambda item: item[0])}
    return sorted_null_p_vals

In [67]:
null_p_vals = get_moa_p_vals(null_distribution_medns, data_moa_vals)

In [68]:
df_null_p_vals = pd.DataFrame.from_dict(null_p_vals, orient='index', 
                                        columns = ["p_value_alldose"]).reset_index().rename(columns={"index": "moa"})

In [69]:
df_null_p_vals = (
    df_null_p_vals
    .merge(data_moa_vals, on="moa")
    .drop(columns=["spearman_correlation"])
    .rename(columns={"no_of_replicates": "moa_size"})
)

In [70]:
df_null_p_vals.shape

(211, 3)

In [71]:
df_null_p_vals.head(10)

,moa,p_value_alldose,moa_size
0,5 alpha reductase inhibitor,1.000,12
1,acat inhibitor,0.988,18
2,acetylcholine receptor agonist,1.000,48
3,acetylcholine receptor antagonist,0.177,120
4,acetylcholine release stimulant,1.000,12
5,acetylcholinesterase inhibitor,0.929,12
6,adenosine receptor agonist,0.964,18
7,adenosine receptor antagonist,0.331,24
8,adrenergic inhibitor,0.157,12
9,adrenergic receptor agonist,0.984,120


In [72]:
def save_to_csv(df, path, file_name):
    """saves moa dataframes to csv"""
    
    if not os.path.exists(path):
        os.mkdir(path)
    
    df.to_csv(os.path.join(path, file_name), index=False)

In [73]:
save_to_csv(df_null_p_vals, 'moa_sizes_consensus_datasets', 'modz_null_p_values_doseindependent.csv')